# 2. Signal Generation

Here we set up the signal generation processes for both $W^+H$ and $W^-H$ separately, so that they can later be analyzed.

In [1]:
import sys
import os

from __future__ import absolute_import, division, print_function, unicode_literals
import logging
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

from madminer.core import MadMiner
from madminer.lhe import LHEReader
from madminer.sampling import SampleAugmenter, combine_and_shuffle
from madminer.plotting import plot_distributions

from scipy.optimize import curve_fit

import madminer.__version__
print( 'MadMiner version: {}'.format(madminer.__version__) )

MadMiner version: 0.3.0


In [2]:
# MadMiner output
logging.basicConfig(
    format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s',
    datefmt='%H:%M',
    level=logging.INFO
)

# Output of all other modules (e.g. matplotlib)
for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

In [3]:
mg_dir = '<eos-path>'

We load the MadMiner morphing setup created in the previous section, which includes morphing benchmarks to span the full parameter space of $(C_{H\square}, C_{HD}, C_{HW}, C_{Hq}^{(3)})$.

In [4]:
miner = MadMiner()
miner.load('data/wh_smeft_setup.h5')

14:42 madminer.core        INFO    Found 4 parameters:
14:42 madminer.core        INFO       cHbox (LHA: frblock 24, maximal power in squared ME: (2,), range: (-10.0, 10.0))
14:42 madminer.core        INFO       cHDD (LHA: frblock 25, maximal power in squared ME: (2,), range: (-10.0, 10.0))
14:42 madminer.core        INFO       cHW (LHA: frblock 28, maximal power in squared ME: (2,), range: (-10.0, 10.0))
14:42 madminer.core        INFO       cHq3 (LHA: frblock 49, maximal power in squared ME: (2,), range: (-10.0, 10.0))
14:42 madminer.core        INFO    Found 15 benchmarks:
14:42 madminer.core        INFO       sm: cHbox = 0.00e+00, cHDD = 0.00e+00, cHW = 0.00e+00, cHq3 = 0.00e+00
14:42 madminer.core        INFO       morphing_basis_vector_1: cHbox = 5.41, cHDD = 5.11, cHW = -5.33e+00, cHq3 = -2.67e+00
14:42 madminer.core        INFO       morphing_basis_vector_2: cHbox = 7.24, cHDD = -2.47e+00, cHW = -7.09e+00, cHq3 = 3.53
14:42 madminer.core        INFO       morphing_basis_vector_

We'll generate a number of samples simulataneously using MadMiner.run_multiple().

A quick summary of what's in the cards used for the generation:

`cards/signal_processes/proc_card_wph_mu_smeftsim.dat`:

    import model SMEFTsim
    generate p p > w+ h > mu+ vm b b~ QCD=0 QED=99 NP=1


`cards/run_card_wh_parton.dat`:

    50000 = nevents
    6500.0   = ebeam1
    6500.0   = ebeam2`
    35.0 = ptb
    10.0 = ptl
    25.0 = misset
    30.0 = ptjmax
    2.5 = etab
    2.5 = etal
    80.0 = mmbb
    160.0 = mmbbmax
    0.4 = drbb
    0.4 = drbl
    True = cut_decays

We generate the events in a number of small batches, using the SM benchmark for sampling.

In [5]:
nrun = 40

## 1. $W^+(\mu^+ \nu_{\mu})H(b\bar{b})$

In [6]:
miner.run_multiple(
    mg_directory=mg_dir,
    log_directory='logs/wph_mu_smeftsim',
    mg_process_directory='./signal_samples/wph_mu_smeftsim',
    proc_card_file='./cards/signal_processes/proc_card_wph_mu_smeftsim.dat',
    param_card_template_file='./cards/param_card_template_smeftsim.dat',
    run_card_files=['./cards/run_card_wh_parton.dat' for i in range(nrun)],
    sample_benchmarks=['sm'],
    initial_command=None
)

14:42 madminer.utils.inter INFO    Generating MadGraph process folder from ../cards/signal_processes/proc_card_wph_mu_smeftsim.dat at ./wph_mu_smeftsim
14:42 madminer.core        INFO    Run 0
14:42 madminer.core        INFO      Sampling from benchmark: sm
14:42 madminer.core        INFO      Original run card:       ../cards/run_card_wh_parton.dat
14:42 madminer.core        INFO      Original Pythia8 card:   None
14:42 madminer.core        INFO      Copied run card:         /madminer/cards/run_card_0.dat
14:42 madminer.core        INFO      Copied Pythia8 card:     None
14:42 madminer.core        INFO      Param card:              /madminer/cards/param_card_0.dat
14:42 madminer.core        INFO      Reweight card:           /madminer/cards/reweight_card_0.dat
14:42 madminer.core        INFO      Log file:                run_0.log
14:42 madminer.core        INFO    Creating param and reweight cards in ./wph_mu_smeftsim//madminer/cards/param_card_0.dat, ./wph_mu_smeftsim//madminer/card

16:35 madminer.core        INFO      Param card:              /madminer/cards/param_card_8.dat
16:35 madminer.core        INFO      Reweight card:           /madminer/cards/reweight_card_8.dat
16:35 madminer.core        INFO      Log file:                run_8.log
16:35 madminer.core        INFO    Creating param and reweight cards in ./wph_mu_smeftsim//madminer/cards/param_card_8.dat, ./wph_mu_smeftsim//madminer/cards/reweight_card_8.dat
16:35 madminer.utils.inter INFO    Starting MadGraph and Pythia in ./wph_mu_smeftsim
16:48 madminer.core        INFO    Run 9
16:48 madminer.core        INFO      Sampling from benchmark: sm
16:48 madminer.core        INFO      Original run card:       ../cards/run_card_wh_parton.dat
16:48 madminer.core        INFO      Original Pythia8 card:   None
16:48 madminer.core        INFO      Copied run card:         /madminer/cards/run_card_9.dat
16:48 madminer.core        INFO      Copied Pythia8 card:     None
16:48 madminer.core        INFO      Param ca

18:37 madminer.core        INFO    Run 17
18:37 madminer.core        INFO      Sampling from benchmark: sm
18:37 madminer.core        INFO      Original run card:       ../cards/run_card_wh_parton.dat
18:37 madminer.core        INFO      Original Pythia8 card:   None
18:37 madminer.core        INFO      Copied run card:         /madminer/cards/run_card_17.dat
18:37 madminer.core        INFO      Copied Pythia8 card:     None
18:37 madminer.core        INFO      Param card:              /madminer/cards/param_card_17.dat
18:37 madminer.core        INFO      Reweight card:           /madminer/cards/reweight_card_17.dat
18:37 madminer.core        INFO      Log file:                run_17.log
18:37 madminer.core        INFO    Creating param and reweight cards in ./wph_mu_smeftsim//madminer/cards/param_card_17.dat, ./wph_mu_smeftsim//madminer/cards/reweight_card_17.dat
18:37 madminer.utils.inter INFO    Starting MadGraph and Pythia in ./wph_mu_smeftsim
18:50 madminer.core        INFO    Run

20:24 madminer.core        INFO      Reweight card:           /madminer/cards/reweight_card_25.dat
20:24 madminer.core        INFO      Log file:                run_25.log
20:24 madminer.core        INFO    Creating param and reweight cards in ./wph_mu_smeftsim//madminer/cards/param_card_25.dat, ./wph_mu_smeftsim//madminer/cards/reweight_card_25.dat
20:24 madminer.utils.inter INFO    Starting MadGraph and Pythia in ./wph_mu_smeftsim
20:38 madminer.core        INFO    Run 26
20:38 madminer.core        INFO      Sampling from benchmark: sm
20:38 madminer.core        INFO      Original run card:       ../cards/run_card_wh_parton.dat
20:38 madminer.core        INFO      Original Pythia8 card:   None
20:38 madminer.core        INFO      Copied run card:         /madminer/cards/run_card_26.dat
20:38 madminer.core        INFO      Copied Pythia8 card:     None
20:38 madminer.core        INFO      Param card:              /madminer/cards/param_card_26.dat
20:38 madminer.core        INFO      R

22:32 madminer.core        INFO      Original run card:       ../cards/run_card_wh_parton.dat
22:32 madminer.core        INFO      Original Pythia8 card:   None
22:32 madminer.core        INFO      Copied run card:         /madminer/cards/run_card_34.dat
22:32 madminer.core        INFO      Copied Pythia8 card:     None
22:32 madminer.core        INFO      Param card:              /madminer/cards/param_card_34.dat
22:32 madminer.core        INFO      Reweight card:           /madminer/cards/reweight_card_34.dat
22:32 madminer.core        INFO      Log file:                run_34.log
22:32 madminer.core        INFO    Creating param and reweight cards in ./wph_mu_smeftsim//madminer/cards/param_card_34.dat, ./wph_mu_smeftsim//madminer/cards/reweight_card_34.dat
22:32 madminer.utils.inter INFO    Starting MadGraph and Pythia in ./wph_mu_smeftsim
22:45 madminer.core        INFO    Run 35
22:45 madminer.core        INFO      Sampling from benchmark: sm
22:45 madminer.core        INFO      O

## 2. $W^+(e^+\nu_{e})H(b\bar{b})$

In [7]:
miner.run_multiple(
    mg_directory=mg_dir,
    log_directory='logs/wph_e_smeftsim',
    mg_process_directory='./signal_samples/wph_e_smeftsim',
    proc_card_file='./cards/signal_processes/proc_card_wph_e_smeftsim.dat',
    param_card_template_file='./cards/param_card_template_smeftsim.dat',
    run_card_files=['./cards/run_card_wh_parton.dat' for i in range(nrun)],
    sample_benchmarks=['sm'],
    initial_command=None
)

23:53 madminer.utils.inter INFO    Generating MadGraph process folder from ../cards/signal_processes/proc_card_wph_e_smeftsim.dat at ./wph_e_smeftsim
23:53 madminer.core        INFO    Run 0
23:53 madminer.core        INFO      Sampling from benchmark: sm
23:53 madminer.core        INFO      Original run card:       ../cards/run_card_wh_parton.dat
23:53 madminer.core        INFO      Original Pythia8 card:   None
23:53 madminer.core        INFO      Copied run card:         /madminer/cards/run_card_0.dat
23:53 madminer.core        INFO      Copied Pythia8 card:     None
23:53 madminer.core        INFO      Param card:              /madminer/cards/param_card_0.dat
23:53 madminer.core        INFO      Reweight card:           /madminer/cards/reweight_card_0.dat
23:53 madminer.core        INFO      Log file:                run_0.log
23:53 madminer.core        INFO    Creating param and reweight cards in ./wph_e_smeftsim//madminer/cards/param_card_0.dat, ./wph_e_smeftsim//madminer/cards/re

01:43 madminer.core        INFO      Param card:              /madminer/cards/param_card_8.dat
01:43 madminer.core        INFO      Reweight card:           /madminer/cards/reweight_card_8.dat
01:43 madminer.core        INFO      Log file:                run_8.log
01:43 madminer.core        INFO    Creating param and reweight cards in ./wph_e_smeftsim//madminer/cards/param_card_8.dat, ./wph_e_smeftsim//madminer/cards/reweight_card_8.dat
01:43 madminer.utils.inter INFO    Starting MadGraph and Pythia in ./wph_e_smeftsim
01:56 madminer.core        INFO    Run 9
01:56 madminer.core        INFO      Sampling from benchmark: sm
01:56 madminer.core        INFO      Original run card:       ../cards/run_card_wh_parton.dat
01:56 madminer.core        INFO      Original Pythia8 card:   None
01:56 madminer.core        INFO      Copied run card:         /madminer/cards/run_card_9.dat
01:56 madminer.core        INFO      Copied Pythia8 card:     None
01:56 madminer.core        INFO      Param card:

03:50 madminer.core        INFO      Sampling from benchmark: sm
03:50 madminer.core        INFO      Original run card:       ../cards/run_card_wh_parton.dat
03:50 madminer.core        INFO      Original Pythia8 card:   None
03:50 madminer.core        INFO      Copied run card:         /madminer/cards/run_card_17.dat
03:50 madminer.core        INFO      Copied Pythia8 card:     None
03:50 madminer.core        INFO      Param card:              /madminer/cards/param_card_17.dat
03:50 madminer.core        INFO      Reweight card:           /madminer/cards/reweight_card_17.dat
03:50 madminer.core        INFO      Log file:                run_17.log
03:50 madminer.core        INFO    Creating param and reweight cards in ./wph_e_smeftsim//madminer/cards/param_card_17.dat, ./wph_e_smeftsim//madminer/cards/reweight_card_17.dat
03:50 madminer.utils.inter INFO    Starting MadGraph and Pythia in ./wph_e_smeftsim
04:03 madminer.core        INFO    Run 18
04:03 madminer.core        INFO      Samp

05:46 madminer.core        INFO      Log file:                run_25.log
05:46 madminer.core        INFO    Creating param and reweight cards in ./wph_e_smeftsim//madminer/cards/param_card_25.dat, ./wph_e_smeftsim//madminer/cards/reweight_card_25.dat
05:46 madminer.utils.inter INFO    Starting MadGraph and Pythia in ./wph_e_smeftsim
06:01 madminer.core        INFO    Run 26
06:01 madminer.core        INFO      Sampling from benchmark: sm
06:01 madminer.core        INFO      Original run card:       ../cards/run_card_wh_parton.dat
06:01 madminer.core        INFO      Original Pythia8 card:   None
06:01 madminer.core        INFO      Copied run card:         /madminer/cards/run_card_26.dat
06:01 madminer.core        INFO      Copied Pythia8 card:     None
06:01 madminer.core        INFO      Param card:              /madminer/cards/param_card_26.dat
06:01 madminer.core        INFO      Reweight card:           /madminer/cards/reweight_card_26.dat
06:01 madminer.core        INFO      Log 

07:57 madminer.core        INFO      Original Pythia8 card:   None
07:57 madminer.core        INFO      Copied run card:         /madminer/cards/run_card_34.dat
07:57 madminer.core        INFO      Copied Pythia8 card:     None
07:57 madminer.core        INFO      Param card:              /madminer/cards/param_card_34.dat
07:57 madminer.core        INFO      Reweight card:           /madminer/cards/reweight_card_34.dat
07:57 madminer.core        INFO      Log file:                run_34.log
07:57 madminer.core        INFO    Creating param and reweight cards in ./wph_e_smeftsim//madminer/cards/param_card_34.dat, ./wph_e_smeftsim//madminer/cards/reweight_card_34.dat
07:57 madminer.utils.inter INFO    Starting MadGraph and Pythia in ./wph_e_smeftsim
08:10 madminer.core        INFO    Run 35
08:10 madminer.core        INFO      Sampling from benchmark: sm
08:10 madminer.core        INFO      Original run card:       ../cards/run_card_wh_parton.dat
08:10 madminer.core        INFO      Orig

## 3. $W^-(\mu^-\bar{\nu}_{\mu})H(b\bar{b})$

In [8]:
miner.run_multiple(
    mg_directory=mg_dir,
    log_directory='logs/wmh_mu_smeftsim',
    mg_process_directory='./signal_samples/wmh_mu_smeftsim',
    proc_card_file='./cards/signal_processes/proc_card_wmh_mu_smeftsim.dat',
    param_card_template_file='./cards/param_card_template_smeftsim.dat',
    run_card_files=['./cards/run_card_wh_parton.dat' for i in range(nrun)],
    sample_benchmarks=['sm'],
    initial_command=None
)

09:18 madminer.utils.inter INFO    Generating MadGraph process folder from ../cards/signal_processes/proc_card_wmh_mu_smeftsim.dat at ./wmh_mu_smeftsim
09:18 madminer.core        INFO    Run 0
09:18 madminer.core        INFO      Sampling from benchmark: sm
09:18 madminer.core        INFO      Original run card:       ../cards/run_card_wh_parton.dat
09:18 madminer.core        INFO      Original Pythia8 card:   None
09:18 madminer.core        INFO      Copied run card:         /madminer/cards/run_card_0.dat
09:18 madminer.core        INFO      Copied Pythia8 card:     None
09:18 madminer.core        INFO      Param card:              /madminer/cards/param_card_0.dat
09:18 madminer.core        INFO      Reweight card:           /madminer/cards/reweight_card_0.dat
09:18 madminer.core        INFO      Log file:                run_0.log
09:18 madminer.core        INFO    Creating param and reweight cards in ./wmh_mu_smeftsim//madminer/cards/param_card_0.dat, ./wmh_mu_smeftsim//madminer/card

11:10 madminer.core        INFO      Param card:              /madminer/cards/param_card_8.dat
11:10 madminer.core        INFO      Reweight card:           /madminer/cards/reweight_card_8.dat
11:10 madminer.core        INFO      Log file:                run_8.log
11:10 madminer.core        INFO    Creating param and reweight cards in ./wmh_mu_smeftsim//madminer/cards/param_card_8.dat, ./wmh_mu_smeftsim//madminer/cards/reweight_card_8.dat
11:10 madminer.utils.inter INFO    Starting MadGraph and Pythia in ./wmh_mu_smeftsim
11:25 madminer.core        INFO    Run 9
11:25 madminer.core        INFO      Sampling from benchmark: sm
11:25 madminer.core        INFO      Original run card:       ../cards/run_card_wh_parton.dat
11:25 madminer.core        INFO      Original Pythia8 card:   None
11:25 madminer.core        INFO      Copied run card:         /madminer/cards/run_card_9.dat
11:25 madminer.core        INFO      Copied Pythia8 card:     None
11:25 madminer.core        INFO      Param ca

13:15 madminer.core        INFO    Run 17
13:15 madminer.core        INFO      Sampling from benchmark: sm
13:15 madminer.core        INFO      Original run card:       ../cards/run_card_wh_parton.dat
13:15 madminer.core        INFO      Original Pythia8 card:   None
13:15 madminer.core        INFO      Copied run card:         /madminer/cards/run_card_17.dat
13:15 madminer.core        INFO      Copied Pythia8 card:     None
13:15 madminer.core        INFO      Param card:              /madminer/cards/param_card_17.dat
13:15 madminer.core        INFO      Reweight card:           /madminer/cards/reweight_card_17.dat
13:15 madminer.core        INFO      Log file:                run_17.log
13:15 madminer.core        INFO    Creating param and reweight cards in ./wmh_mu_smeftsim//madminer/cards/param_card_17.dat, ./wmh_mu_smeftsim//madminer/cards/reweight_card_17.dat
13:15 madminer.utils.inter INFO    Starting MadGraph and Pythia in ./wmh_mu_smeftsim
13:29 madminer.core        INFO    Run

15:04 madminer.core        INFO      Reweight card:           /madminer/cards/reweight_card_25.dat
15:04 madminer.core        INFO      Log file:                run_25.log
15:04 madminer.core        INFO    Creating param and reweight cards in ./wmh_mu_smeftsim//madminer/cards/param_card_25.dat, ./wmh_mu_smeftsim//madminer/cards/reweight_card_25.dat
15:04 madminer.utils.inter INFO    Starting MadGraph and Pythia in ./wmh_mu_smeftsim
15:18 madminer.core        INFO    Run 26
15:18 madminer.core        INFO      Sampling from benchmark: sm
15:18 madminer.core        INFO      Original run card:       ../cards/run_card_wh_parton.dat
15:18 madminer.core        INFO      Original Pythia8 card:   None
15:18 madminer.core        INFO      Copied run card:         /madminer/cards/run_card_26.dat
15:18 madminer.core        INFO      Copied Pythia8 card:     None
15:18 madminer.core        INFO      Param card:              /madminer/cards/param_card_26.dat
15:18 madminer.core        INFO      R

17:08 madminer.core        INFO      Original run card:       ../cards/run_card_wh_parton.dat
17:08 madminer.core        INFO      Original Pythia8 card:   None
17:08 madminer.core        INFO      Copied run card:         /madminer/cards/run_card_34.dat
17:08 madminer.core        INFO      Copied Pythia8 card:     None
17:08 madminer.core        INFO      Param card:              /madminer/cards/param_card_34.dat
17:08 madminer.core        INFO      Reweight card:           /madminer/cards/reweight_card_34.dat
17:08 madminer.core        INFO      Log file:                run_34.log
17:08 madminer.core        INFO    Creating param and reweight cards in ./wmh_mu_smeftsim//madminer/cards/param_card_34.dat, ./wmh_mu_smeftsim//madminer/cards/reweight_card_34.dat
17:08 madminer.utils.inter INFO    Starting MadGraph and Pythia in ./wmh_mu_smeftsim
17:21 madminer.core        INFO    Run 35
17:21 madminer.core        INFO      Sampling from benchmark: sm
17:21 madminer.core        INFO      O

## 4. $W^-(e^-\bar{\nu}_{e})H(b\bar{b})$

In [9]:
miner.run_multiple(
    mg_directory=mg_dir,
    log_directory='logs/wmh_e_smeftsim',
    mg_process_directory='./signal_samples/wmh_e_smeftsim',
    proc_card_file='./cards/signal_processes/proc_card_wmh_e_smeftsim.dat',
    param_card_template_file='./cards/param_card_template_smeftsim.dat',
    run_card_files=['./cards/run_card_wh_parton.dat' for i in range(nrun)],
    sample_benchmarks=['sm'],
    initial_command=None
)

18:28 madminer.utils.inter INFO    Generating MadGraph process folder from ../cards/signal_processes/proc_card_wmh_e_smeftsim.dat at ./wmh_e_smeftsim
18:28 madminer.core        INFO    Run 0
18:28 madminer.core        INFO      Sampling from benchmark: sm
18:28 madminer.core        INFO      Original run card:       ../cards/run_card_wh_parton.dat
18:28 madminer.core        INFO      Original Pythia8 card:   None
18:28 madminer.core        INFO      Copied run card:         /madminer/cards/run_card_0.dat
18:28 madminer.core        INFO      Copied Pythia8 card:     None
18:28 madminer.core        INFO      Param card:              /madminer/cards/param_card_0.dat
18:28 madminer.core        INFO      Reweight card:           /madminer/cards/reweight_card_0.dat
18:28 madminer.core        INFO      Log file:                run_0.log
18:28 madminer.core        INFO    Creating param and reweight cards in ./wmh_e_smeftsim//madminer/cards/param_card_0.dat, ./wmh_e_smeftsim//madminer/cards/re

20:31 madminer.core        INFO      Param card:              /madminer/cards/param_card_8.dat
20:31 madminer.core        INFO      Reweight card:           /madminer/cards/reweight_card_8.dat
20:31 madminer.core        INFO      Log file:                run_8.log
20:31 madminer.core        INFO    Creating param and reweight cards in ./wmh_e_smeftsim//madminer/cards/param_card_8.dat, ./wmh_e_smeftsim//madminer/cards/reweight_card_8.dat
20:31 madminer.utils.inter INFO    Starting MadGraph and Pythia in ./wmh_e_smeftsim
20:46 madminer.core        INFO    Run 9
20:46 madminer.core        INFO      Sampling from benchmark: sm
20:46 madminer.core        INFO      Original run card:       ../cards/run_card_wh_parton.dat
20:46 madminer.core        INFO      Original Pythia8 card:   None
20:46 madminer.core        INFO      Copied run card:         /madminer/cards/run_card_9.dat
20:46 madminer.core        INFO      Copied Pythia8 card:     None
20:46 madminer.core        INFO      Param card:

22:46 madminer.core        INFO      Sampling from benchmark: sm
22:46 madminer.core        INFO      Original run card:       ../cards/run_card_wh_parton.dat
22:46 madminer.core        INFO      Original Pythia8 card:   None
22:46 madminer.core        INFO      Copied run card:         /madminer/cards/run_card_17.dat
22:46 madminer.core        INFO      Copied Pythia8 card:     None
22:46 madminer.core        INFO      Param card:              /madminer/cards/param_card_17.dat
22:46 madminer.core        INFO      Reweight card:           /madminer/cards/reweight_card_17.dat
22:46 madminer.core        INFO      Log file:                run_17.log
22:46 madminer.core        INFO    Creating param and reweight cards in ./wmh_e_smeftsim//madminer/cards/param_card_17.dat, ./wmh_e_smeftsim//madminer/cards/reweight_card_17.dat
22:46 madminer.utils.inter INFO    Starting MadGraph and Pythia in ./wmh_e_smeftsim
23:01 madminer.core        INFO    Run 18
23:01 madminer.core        INFO      Samp

00:40 madminer.core        INFO      Log file:                run_25.log
00:40 madminer.core        INFO    Creating param and reweight cards in ./wmh_e_smeftsim//madminer/cards/param_card_25.dat, ./wmh_e_smeftsim//madminer/cards/reweight_card_25.dat
00:40 madminer.utils.inter INFO    Starting MadGraph and Pythia in ./wmh_e_smeftsim
00:55 madminer.core        INFO    Run 26
00:55 madminer.core        INFO      Sampling from benchmark: sm
00:55 madminer.core        INFO      Original run card:       ../cards/run_card_wh_parton.dat
00:55 madminer.core        INFO      Original Pythia8 card:   None
00:55 madminer.core        INFO      Copied run card:         /madminer/cards/run_card_26.dat
00:55 madminer.core        INFO      Copied Pythia8 card:     None
00:55 madminer.core        INFO      Param card:              /madminer/cards/param_card_26.dat
00:55 madminer.core        INFO      Reweight card:           /madminer/cards/reweight_card_26.dat
00:55 madminer.core        INFO      Log 

02:53 madminer.core        INFO      Original Pythia8 card:   None
02:53 madminer.core        INFO      Copied run card:         /madminer/cards/run_card_34.dat
02:53 madminer.core        INFO      Copied Pythia8 card:     None
02:53 madminer.core        INFO      Param card:              /madminer/cards/param_card_34.dat
02:53 madminer.core        INFO      Reweight card:           /madminer/cards/reweight_card_34.dat
02:53 madminer.core        INFO      Log file:                run_34.log
02:53 madminer.core        INFO    Creating param and reweight cards in ./wmh_e_smeftsim//madminer/cards/param_card_34.dat, ./wmh_e_smeftsim//madminer/cards/reweight_card_34.dat
02:53 madminer.utils.inter INFO    Starting MadGraph and Pythia in ./wmh_e_smeftsim
03:08 madminer.core        INFO    Run 35
03:08 madminer.core        INFO      Sampling from benchmark: sm
03:08 madminer.core        INFO      Original run card:       ../cards/run_card_wh_parton.dat
03:08 madminer.core        INFO      Orig